# Graph-based Adversarial Machine Learning

# Import Libraries

In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torch-geometric pywavelets

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch.optim import Adam
import numpy as np

# Load Dataset

In [22]:
# Load the dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define Graph Neural Network

In [23]:
# Define the GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_features, num_classes, heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, 16, heads=heads, dropout=0.6)
        self.conv2 = GATConv(16 * heads, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train Model

In [25]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Get Accuracy of Model

In [26]:
# Evaluate the model
def evaluate(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
    accuracy = correct / data.test_mask.sum().item()
    return accuracy

accuracy_before = evaluate(model, data)
print(f'Accuracy before attacks: {accuracy_before:.4f}')

Accuracy before attacks: 0.8130


# FGSM Attack

In [27]:
# FGSM Attack
def fgsm_attack(model, data, epsilon):
    data.x.requires_grad = True
    output = model(data)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    model.zero_grad()
    loss.backward()
    data_grad = data.x.grad.data
    sign_data_grad = data_grad.sign()
    perturbed_data = data.x + epsilon * sign_data_grad
    perturbed_data = torch.clamp(perturbed_data, 0, 1)
    return perturbed_data

epsilon = 0.1
data_fgsm = data.clone()
data_fgsm.x = fgsm_attack(model, data, epsilon)
accuracy_after_fgsm = evaluate(model, data_fgsm)
print(f'Accuracy after FGSM attack: {accuracy_after_fgsm:.4f}')

Accuracy after FGSM attack: 0.1360


# PGD Attack

In [28]:
def pgd_attack(model, data, epsilon, alpha, num_iter):
    perturbed_data = data.x.clone().detach().requires_grad_(True)
    for _ in range(num_iter):
        data.x = perturbed_data  # Use perturbed_data in the forward pass
        output = model(data)
        loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
        model.zero_grad()
        loss.backward()
        data_grad = perturbed_data.grad.data
        perturbed_data = perturbed_data + alpha * data_grad.sign()
        perturbation = torch.clamp(perturbed_data - data.x, -epsilon, epsilon)
        perturbed_data = torch.clamp(data.x + perturbation, 0, 1)
        perturbed_data = perturbed_data.detach().requires_grad_(True)
    return perturbed_data

alpha = 0.01
num_iter = 40
data_pgd = data.clone()
data_pgd.x = pgd_attack(model, data, epsilon, alpha, num_iter)
accuracy_after_pgd = evaluate(model, data_pgd)
print(f'Accuracy after PGD attack: {accuracy_after_pgd:.4f}')

Accuracy after PGD attack: 0.2520


# Carlini & Wagner (C&W) Attack

In [29]:
# Carlini & Wagner (C&W) Attack
def cw_attack(model, data, c=1e-4, lr=0.01, num_iter=1000):
    data_adv = data.clone()
    delta = torch.zeros_like(data.x, requires_grad=True).to(device)
    optimizer = Adam([delta], lr=lr)

    for _ in range(num_iter):
        optimizer.zero_grad()
        adv_data = data.x + delta
        output = model(data)
        loss1 = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
        loss2 = c * torch.norm(delta, p=2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        delta.data = torch.clamp(data.x + delta.data, 0, 1) - data.x
    data_adv.x = torch.clamp(data.x + delta.data, 0, 1)
    return data_adv.x

data_cw = data.clone()
data_cw.x = cw_attack(model, data)
accuracy_after_cw = evaluate(model, data_cw)
print(f'Accuracy after C&W attack: {accuracy_after_cw:.4f}')



Accuracy after C&W attack: 0.2520
